In [27]:
import taikoParseRest,json 


In [28]:
taikoAPI = taikoParseRest.TaikoRestAPI(_ApplicationId = "anYw5Fvt4ZQhTN35QnhbYNJ7MLirWYii3BRPFQP7",
                                       _MasterKey = "sKGezpJ8YZ9IFWf7CQOF87wvXFulQ3p6QcAom7oR")

In [6]:
import requests, json, os
import imageio
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import cv2
import datetime
import time

cascPath = './FaceDetect/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)
headers = {
    'authority': 'www.instagram.com',
    'accept': '*/*',
    'x-ig-www-claim': 'hmac.AR1iQ9S27y_K4AbcHrujYM8peOsso-NFjKPe5C2HhPkXM2o9',
    'x-requested-with': 'XMLHttpRequest',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
    'x-csrftoken': 'MlARdprl45bMxHApN0JoUDaTDZ9R8EzX',
    'x-ig-app-id': '936619743392459',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.instagram.com/lilyvaranya/followers/',
    'accept-language': 'en-US,en;q=0.9',   
    #'cookie': 'sessionid=37798735497%3Atef3KwIJVNW14N%3A6;',
    #'cookie': 'sessionid=5813920265%3AYWeSQ0in8Fk9jA%3A1;',
}

_DELAY = 3

proxies = {
    'http':'socks5h://localhost:9050',
    'https': 'socks5h://localhost:9050'
}

faceProto="opencv_face_detector.pbtxt"
faceModel="opencv_face_detector_uint8.pb"
ageProto="age_deploy.prototxt"
ageModel="age_net.caffemodel"
genderProto="gender_deploy.prototxt"
genderModel="gender_net.caffemodel"



In [11]:
def getPostCommenterProfilePictureUrls(userId):
    getFeedDataSuccesses = []
    getFeedDataTimes = []
    getFeedDataLens = []
    
    getCommentProfilePictureUrlSuccesses = []
    getCommentProfilePictureUrlTimes = []
    getCommentLens = []
    
    
    start_connection = time.time()
    skip_user = 0
    print(f"getPostCommenterProfilePictureUrls userId: {userId}")
    
    # Get images
    url_set = set()
    targetPicNumber = 1000
    
    after = ''
    getFail = False
    # Connect to feed
    while True:
        #feedURL = "response = requests.get('https://www.instagram.com/graphql/query/', headers=headers, params = params)"
        graphQL = 'https://www.instagram.com/graphql/query/'
        
        end_cursor = ''
        data = {
                        "id":userId,
                        "first":50,
                        "after":after}
        params = {'query_hash':'bfa387b2992c3a52dcbe447467b4b771'}
        params['variables']  = json.dumps(data, indent = 4)
        print(f"Delay {_DELAY} seconds 1")
        time.sleep(_DELAY)
        start_getFeedDataTime = time.time() ##Boss Check
        try:
            response = requests.get('https://www.instagram.com/graphql/query/', headers = headers, params = params, proxies = proxies)
        except KeyboardInterrupt:
            raise
        except Exception as e:
            print('continue 1', e)
            continue
        print('Convert feedData....')
        try:
            feedData = response.json()
        except json.JSONDecodeError:
            print("Fail connected")
            print(f"Delay {_DELAY} seconds 2")
            #time.sleep(_DELAY)
            continue
        end_getFeedDataTime = time.time() ##Boss Check
        getFeedDataTimes.append(end_getFeedDataTime - start_getFeedDataTime) ##Boss Check
        #
        if feedData.get('status') == 'fail':
            getFeedDataSuccesses.append(0) ##Boss Check
            getFail = True
            print("status fail")
            print(f"Delay {_DELAY} seconds 3")
            if skip_user >= 20:
                break
            else:
                skip_user += 1
                print(f"Fail count: {skip_user}")
            #time.sleep(_DELAY)
            continue
        if not feedData['data']['user']:
            getFeedDataSuccesses.append(0) ##Boss Check
            skip += 1
            continue
        
        getFeedDataSuccesses.append(1) ##Boss Check
        getFeedDataLens.append(len(feedData['data']['user']['edge_owner_to_timeline_media']['edges'])) ##Boss Check
        for post in feedData['data']['user']['edge_owner_to_timeline_media']['edges']:
            param_comments = {"query_hash" : "bc3296d1ce80a24b1b6e40b1e72903f5"}
            # print(post['node']['shortcode'])
            variables_comments = {"shortcode":post['node']['shortcode'],
                                  "first":50,
                                 }
            param_comments['variables']  = json.dumps(variables_comments, indent = 4)
            start_getCommentProfilePictureUrlTime = time.time() ##Boss Check
            print('Getting Post Comment....')
            for _ in range(100):
                try:
                    response_comments = requests.get('https://www.instagram.com/graphql/query/',  params = param_comments, proxies = proxies)
                    break
                except KeyboardInterrupt:
                    raise
                except:
                    if _ == 99:
                        continue
            end_getCommentProfilePictureUrlTime = time.time() ##Boss Check
            getCommentProfilePictureUrlTimes.append(end_getCommentProfilePictureUrlTime - start_getCommentProfilePictureUrlTime) ##Boss Check
            try: 
                data_comment = response_comments.json()
            except json.JSONDecodeError:
                continue
            if data_comment.get('status') == 'fail':
                getCommentProfilePictureUrlSuccesses.append(0) ##Boss Check
                getFail = False
                continue
            
            getCommentProfilePictureUrlSuccesses.append(1) ##Boss Check
            getCommentLens.append(len(data_comment['data']['shortcode_media']['edge_media_to_parent_comment']['edges'])) ##Boss Check
            print('Storing profile picture....')
            for comment in data_comment['data']['shortcode_media']['edge_media_to_parent_comment']['edges']:
                #print(comment['node']['owner']['profile_pic_url'])
                url_set.update({comment['node']['owner']['profile_pic_url']})
            print('len url_ser', len(url_set))
            
        hasNextPage = feedData['data']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
        finish = len(url_set) >= targetPicNumber
        if finish:
            break
        if not hasNextPage:
            break
        else:
            if getFail:
                break
            print("Continue getting")
            print(len(url_set))
            after = feedData['data']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
    
    end_connection = time.time()
    print(f"Connection time {end_connection - start_connection}")
    
    return url_set, getFeedDataSuccesses, getFeedDataTimes, getFeedDataLens, getCommentProfilePictureUrlSuccesses, getCommentProfilePictureUrlTimes, getCommentLens 

In [12]:
a,b,c,d,e,f,g = getPostCommenterProfilePictureUrls('9595407')

getPostCommenterProfilePictureUrls userId: 9595407
Delay 3 seconds 1
Convert feedData....
Getting Post Comment....
Storing profile picture....
len url_ser 48
Getting Post Comment....
Storing profile picture....
len url_ser 88
Getting Post Comment....
Storing profile picture....
len url_ser 113
Getting Post Comment....
Storing profile picture....
len url_ser 139
Getting Post Comment....
Storing profile picture....
len url_ser 168
Getting Post Comment....
Storing profile picture....
len url_ser 206
Getting Post Comment....
Storing profile picture....
len url_ser 236
Getting Post Comment....
Storing profile picture....
len url_ser 274
Getting Post Comment....
Storing profile picture....
len url_ser 310
Getting Post Comment....
Storing profile picture....
len url_ser 351
Getting Post Comment....
Storing profile picture....
len url_ser 379
Getting Post Comment....
Storing profile picture....
len url_ser 408
Getting Post Comment....
Storing profile picture....
len url_ser 443
Getting Post Co

In [5]:
def get_tor_session():
    session = requests.session()
    # Tor uses the 9050 port as the default socks port
    session.proxies = {'http':  'socks5h://127.0.0.1:9050',
                   'https': 'socks5h://127.0.0.1:9050'}
    return session

# Make a request through the Tor connection
# IP visible through Tor
session = get_tor_session()
print(session.get("http://httpbin.org/ip").text)
# Above should print an IP different than your public IP

# Following prints your normal public IP
print(requests.get("http://httpbin.org/ip").text)

{
  "origin": "178.20.55.18"
}

{
  "origin": "124.122.197.77"
}



In [ ]:
skip = 0
for __ in range(100):
    start = time.time()
    start_connection = time.time()
    try:
        del objectId
    except NameError:
        pass
    for _ in range(10):
        try:
            res = taikoAPI.get(field = 'ExternalSocialAccount', 
                               socialType = 1,
                               follower = {'$gt':10000},
                               isThai = True,
                               isShop = False,
                               insight = {"$exists":False},
                               isPrivate = False,
                               order = 'updatedAt',
                               keys="objectId,socialAccountId,isThai,totalFollower",
                               skip = skip)
            break
        except json.JSONDecodeError:
            if _ == 9:
                raise
        except KeyboardInterrupt:
            raise
    skip_user = 0
    userId = res['results'][0]['socialAccountId']
    objectId = res['results'][0]['objectId']
    totalFollower = res['results'][0]['totalFollower']
    print(f"objectId: {objectId}")
    
    # Get images
    url_set = set()
    targetPicNumber = 1000
    
    userId = res['results'][0]['socialAccountId']
    after = ''
    getFail = False
    # Connect to feed
    while True:
        #feedURL = "response = requests.get('https://www.instagram.com/graphql/query/', headers=headers, params = params)"
        graphQL = 'https://www.instagram.com/graphql/query/'
        
        end_cursor = ''
        data = {
                        "id":userId,
                        "first":50,
                        "after":after}
        params = {'query_hash':'bfa387b2992c3a52dcbe447467b4b771'}
        params['variables']  = json.dumps(data, indent = 4)
        print(f"Delay {_DELAY} seconds")
        time.sleep(_DELAY)
        try:
            response = requests.get('https://www.instagram.com/graphql/query/', headers = headers, params = params, proxies = proxies)
        except KeyboardInterrupt:
            raise
        except:
            continue
        try:
            feedData = response.json()
        except json.JSONDecodeError:
            print("Fail connected")
            print(f"Delay {_DELAY} seconds")
            #time.sleep(_DELAY)
            continue
        #
        if feedData.get('status') == 'fail':
            getFail = True
            print("status fail")
            print(f"Delay {_DELAY} seconds")
            if skip_user >= 20:
                break
            else:
                skip_user += 1
                print(f"Fail count: {skip_user}")
            #time.sleep(_DELAY)
            continue
        if not feedData['data']['user']:
            skip += 1
            continue
        for post in feedData['data']['user']['edge_owner_to_timeline_media']['edges']:
            param_comments = {"query_hash" : "bc3296d1ce80a24b1b6e40b1e72903f5"}
            # print(post['node']['shortcode'])
            variables_comments = {"shortcode":post['node']['shortcode'],
                                  "first":50,
                                 }
            param_comments['variables']  = json.dumps(variables_comments, indent = 4)
            for _ in range(100):
                try:
                    response_comments = requests.get('https://www.instagram.com/graphql/query/',  params = param_comments, proxies = proxies)
                    break
                except KeyboardInterrupt:
                    raise
                except:
                    if _ == 99:
                        continue
            try:
                data_comment = response_comments.json()
            except json.JSONDecodeError:
                continue
            if data_comment.get('status') == 'fail':
                getFail = False
                continue
            for comment in data_comment['data']['shortcode_media']['edge_media_to_parent_comment']['edges']:
                #print(comment['node']['owner']['profile_pic_url'])
                url_set.update({comment['node']['owner']['profile_pic_url']})

        hasNextPage = feedData['data']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
        finish = len(url_set) >= targetPicNumber
        if finish:
            break
        if not hasNextPage:
            break
        else:
            if getFail:
                break
            print("Continue getting")
            print(len(url_set))
            after = feedData['data']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
    if skip_user >= 20:
        skip +1 
        continue
    if getFail:
        print("Getting comments is not completed")
        #continue
    end_connection = time.time()
    print(f"Connection time {end_connection - start_connection}")
    start_analyse = time.time()
    print(f"N sample : {len(url_set)}")
    genderList=['Male','Female','Other']
    gender_result = {
            'Male':0,
            'Female':0,
            'Other':0
    }
    age_result = {

    }

    age_gender_result = {
        'Male':{

        },
        'Female':{

        },
        'Other':{

        }
    }
    ageList=['(0-4)',
     '(4-8)',
     '(8-15)',
     '(15-25)',
     '(25-38)',
     '(38-48)',
     '(48-60)',
     '(60-100)']
    ageDirList = [
        '0-17',
        '18-25',
        '26-40',
        '40-59',
        '60+'
    ]
    # Get insights
    MODEL_MEAN_VALUES=(78.4263377603, 87.7689143744, 114.895847746, 140)
    faceNet=cv2.dnn.readNet(faceModel,faceProto)
    ageNet=cv2.dnn.readNet(ageModel,ageProto)
    genderNet=cv2.dnn.readNet(genderModel,genderProto)
    count = 0
    try:
        os.mkdir(f'output/{userId}')
        print(f"Created output results of {userId}")
    except:
        print("Can't create folders")
    for i,picUrl in enumerate(url_set):
        print(f"Index: {i}")
        try:
            image = imageio.imread(picUrl)
    #         plt.imshow(image)
    #         plt.show()
        except KeyboardInterrupt:
            raise
        except:
            print("Can't connect to the image")
            continue
        if image is None:
            print("Image not found")
            continue
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        try:
            image = cv2.resize(image,(1000,1000))
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        except KeyboardInterrupt:
            raise
        except:
            print("Can't convert the image")
            continue
        # Detect faces in the image
        faces = faceCascade.detectMultiScale(
            gray,
            scaleFactor=1.3,
            minNeighbors=20,
            minSize=(100, 100),
        )

        if len(faces) != 0:
            #print(f"Get {followerId}")
            count += 1
            print(f'Face found count: {count}')
            print(f"Found {len(faces)} faces!")

            # Draw a rectangle around the faces
            maxArea = 0
            for (x, y, w, h) in faces:
                if w*h > maxArea:
                    face = cv2.resize(image[y:y+h,x:x+w],(1000,1000))
                    maxArea = w*h
                    #cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 4)

            # print(f'x: {x}, y: {y}, w: {w}, h: {h}')
            blob=cv2.dnn.blobFromImage(face, 1.0, (227,227), MODEL_MEAN_VALUES, swapRB=False)

            genderNet.setInput(blob)
            genderPreds = genderNet.forward()
            model_1_gender = genderList[genderPreds[0].argmax()]
            model_1_gender_confidence = max(genderPreds[0])*100
            print(f'Model 1 Gender: {model_1_gender} confidence level: {model_1_gender_confidence} %')

            gender_out = model_1_gender

            gender_result[gender_out] += 1
            ageNet.setInput(blob)
            agePreds=ageNet.forward()
            result_age = dict(zip(ageList,agePreds[0]))

            age=ageList[agePreds[0].argmax()]
            if age in ['(0-4)', '(4-8)','(8-15)']:
                age = '13-17'
            elif age in ['(15-25)']:
                age = '18-24'
            elif age in ['(25-38)']:
                age = '25-34'
            elif age in ['(38-48)']:# ,'(48-60)']:
                age = '35-44'
            elif age in ['(48-60)']:
                age = '45-60'
            elif age in ['(60-100)']:
                age = '60+'
            conf = max(agePreds[0])*100
            print(f'Age: {age} years confidence level: {conf} %')

            if conf >= 65:
                try:
                    age_result[age] += 1
                except KeyError:
                    age_result[age] = 1
                try:
                    age_gender_result[gender_out][age] += 1
                except KeyError:
                    age_gender_result[gender_out][age] = 1
            imageio.imwrite(f"output/{userId}/{i}_{gender_out}_{age}.jpg",image)
    # aggregate
    sum_gender = sum(gender_result.values()) or 1
    gender_result_object = {k:(v/sum_gender)*100 for k,v in gender_result.items()}

    sum_age_male = sum(age_gender_result['Male'].values()) or 1
    sum_age_female = sum(age_gender_result['Female'].values()) or 1
    sum_age_other = sum(age_gender_result['Other'].values()) or 1

    age_gender_male_object = {k:(v/sum_age_male)*100 for k,v in age_gender_result['Male'].items()}
    age_gender_female_object = {k:(v/sum_age_female)*100 for k,v in age_gender_result['Female'].items()}
    age_gender_other_object = {k:(v/sum_age_other)*100 for k,v in age_gender_result['Other'].items()}

    sum_age = sum(age_result.values()) or 1
    age_result_object = {k:(v/sum_age)*100 for k,v in age_result.items()}
    try:
        for _ in range(15):
            taikoAPI.update(field = 'ExternalSocialAccount',
                    objectId=objectId, 
                    data={
                        "insight":insight,
                        "insightLastUpdatedAt":{
                            "__type": "Date", 
                            "iso": datetime.datetime.now().isoformat()
                        }
                    }
            )
            break
    except KeyboardInterrupt:
        raise
    except:
        pass
    insight = {"audience":{
        "N":count,
        "totalFollower":totalFollower,
        "timestamp":{
                            "__type": "Date", 
                            "iso": datetime.datetime.now().isoformat()
                        },
        "gender":gender_result_object,
        "age":age_result_object,
        "gender_age":{
            "Male":age_gender_male_object,
            "Female":age_gender_female_object,
            "Other":age_gender_other_object
        }
    }}
    end_analyse = time.time()
    print(f"Connection time {end_connection - start_connection}")
    print(f"Analyse tiem: {end_analyse - start_analyse}")
    end = time.time()
    print(f"time usage: {end - start}")
    print("=======================================")